# 03 – Entropic Embeddings and Cognitive Distances
This notebook derives symbolic embeddings from the SWOW graph and connects them to the symbolic manifold $(\alpha, \kappa, E_r)$. It uses network embeddings, entropy measures, and topological proximity to operationalize cognitive curvature and symbolic unpredictability.

In [ ]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pathlib

from node2vec import Node2Vec
from sklearn.decomposition import PCA
import seaborn as sns

# Load graph
graph_path = pathlib.Path("../data/swow_graph.gpickle")
G = nx.read_gpickle(graph_path)
print(f"Graph loaded: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

## Embedding the Graph with Node2Vec
We use Node2Vec to learn continuous feature representations for each node based on structural similarity and proximity.

In [ ]:
# Run Node2Vec
node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=2)
model = node2vec.fit(window=10, min_count=1)

# Create embedding dataframe
embedding_df = pd.DataFrame([model.wv[node] for node in G.nodes()], index=list(G.nodes()))
embedding_df.head()

## Dimensionality Reduction
To visualize and use embeddings in symbolic models, we reduce them to 2D or 3D using PCA.

In [ ]:
# PCA projection
pca = PCA(n_components=2)
coords = pca.fit_transform(embedding_df)
embedding_df[["x", "y"]] = coords

sns.scatterplot(data=embedding_df, x="x", y="y", alpha=0.3)
plt.title("Node Embeddings (PCA projection)")
plt.show()

## Cognitive Symbolic Metrics (Curvature and Entropy)
We estimate local entropy and divergence using neighbors of each node.

In [ ]:
def local_entropy(node):
    neighbors = list(G.neighbors(node))
    degs = np.array([G.degree(n) for n in neighbors])
    if len(degs) == 0:
        return 0
    p = degs / degs.sum()
    return -(p * np.log2(p)).sum()

embedding_df['E_r'] = embedding_df.index.to_series().apply(local_entropy)